In [ ]:
import os
import shutil
import json
import yaml

In [ ]:
path_to_perf_co3d = "/home/yoonwoo/data/perfception/perfception_co3d"

In [ ]:
cnt = 0
co3d_lists = os.listdir(path_to_perf_co3d)
for co3d_dir_name in co3d_lists:
    co3d_dir_path = os.path.join(path_to_perf_co3d, co3d_dir_name)
    results_path = os.path.join(co3d_dir_path, "results.json")
    if not os.path.exists(results_path):
        # shutil.rmtree(co3d_dir_path)
        cnt += 1
print(cnt, len(co3d_lists))

In [ ]:
with open("../../dataloader/co3d_lists/co3d_list.json") as fp:
    co3d_all_lists = json.load(fp)

co3d_lists = os.listdir(path_to_perf_co3d)
print(co3d_lists[0], list(co3d_all_lists.keys())[0])
co3d_missing_scene_list = []
for co3d_dir_name in co3d_all_lists.keys():
    if "plenoxel_co3d_" + co3d_dir_name not in co3d_lists:
        co3d_missing_scene_list.append(co3d_dir_name)

def generate_sweep_co3d():
    params = dict()
    params["scene_name"] = dict(values=co3d_missing_scene_list)
    params["entity"] = dict(value="postech_cvlab")
    config = dict()
    config["method"] = "grid"
    config["program"] = "run.py"
    config["parameters"] = params
    with open("../../cache/co3d_missing.yaml", "w") as fp:
        yaml.dump(config, fp)

generate_sweep_co3d()